In [1]:
import requests
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time

In [2]:
API_URL = "http://localhost:8000"
EXPERIMENT_NAME = "ad_campaign_test"
ARMS = ["Ad_Creative_A", "Ad_Creative_B", "Ad_Creative_C"]
TRUE_CONVERSION_RATES = [0.05, 0.08, 0.12]  # True CTR for each ad
N_ITERATIONS = 1000

In [3]:
def create_experiment(algorithm="epsilon_greedy", epsilon=0.1):
    response = requests.post(
        f"{API_URL}/experiments",
        json={
            "experiment_name": EXPERIMENT_NAME,
            "arms": ARMS,
            "algorithm": algorithm,
            "epsilon": epsilon
        }
    )
    return response.json()

In [4]:
def simulate_interaction():
    # Select arm
    select_response = requests.post(
        f"{API_URL}/select",
        json={"experiment_name": EXPERIMENT_NAME}
    )
    selection = select_response.json()
    
    # Simulate conversion (Bernoulli reward)
    arm_idx = selection["arm_index"]
    conversion_rate = TRUE_CONVERSION_RATES[arm_idx]
    reward = float(np.random.random() < conversion_rate)
    
    # Update reward
    requests.post(
        f"{API_URL}/update",
        json={
            "experiment_name": EXPERIMENT_NAME,
            "arm_index": arm_idx,
            "reward": reward
        }
    )
    
    return arm_idx, reward

In [6]:
print("Creating experiment...")
create_experiment()

Creating experiment...


{'message': "Experiment 'ad_campaign_test' created successfully"}

In [7]:
print(f"Running {N_ITERATIONS} simulations...")
results = []

Running 1000 simulations...


In [8]:
for i in range(N_ITERATIONS):
    arm, reward = simulate_interaction()
    
    # Get stats every 50 iterations
    if (i + 1) % 50 == 0:
        stats_response = requests.get(
            f"{API_URL}/experiments/{EXPERIMENT_NAME}/stats"
        )
        stats = stats_response.json()
        results.append({
            "iteration": i + 1,
            **stats
        })
        print(f"Iteration {i+1}: Avg Reward = {stats['average_reward']:.4f}")

Iteration 50: Avg Reward = 0.0800
Iteration 100: Avg Reward = 0.0700
Iteration 150: Avg Reward = 0.0933
Iteration 200: Avg Reward = 0.0750
Iteration 250: Avg Reward = 0.0720
Iteration 300: Avg Reward = 0.0800
Iteration 350: Avg Reward = 0.0800
Iteration 400: Avg Reward = 0.0775
Iteration 450: Avg Reward = 0.0822
Iteration 500: Avg Reward = 0.0800
Iteration 550: Avg Reward = 0.0836
Iteration 600: Avg Reward = 0.0800
Iteration 650: Avg Reward = 0.0846
Iteration 700: Avg Reward = 0.0914
Iteration 750: Avg Reward = 0.0933
Iteration 800: Avg Reward = 0.0938
Iteration 850: Avg Reward = 0.0941
Iteration 900: Avg Reward = 0.0967
Iteration 950: Avg Reward = 0.0989
Iteration 1000: Avg Reward = 0.1000


In [9]:
df = pd.DataFrame(results)

In [10]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Average Reward Over Time',
        'Arm Selection Distribution',
        'Arm Value Estimates',
        'Cumulative Pulls per Arm'
    )
)

In [ ]:
fig.add_trace(
    go.Scatter(x=df['iteration'], y=df['average_reward'], name='Avg Reward'),
    row=1, col=1
)
fig.add_hline(y=max(TRUE_CONVERSION_RATES), line_dash="dash", 
              annotation_text="Optimal", row=1, col=1)

In [12]:
final_stats = results[-1]
fig.add_trace(
    go.Bar(x=ARMS, y=final_stats['arm_counts'], name='Pulls'),
    row=1, col=2
)

In [13]:
fig.add_trace(
    go.Bar(x=ARMS, y=final_stats['arm_values'], name='Estimated Value'),
    row=2, col=1
)
fig.add_trace(
    go.Bar(x=ARMS, y=TRUE_CONVERSION_RATES, name='True Value', 
           marker_color='red', opacity=0.5),
    row=2, col=1
)

In [14]:
for i, arm in enumerate(ARMS):
    pulls = [r['arm_counts'][i] for r in results]
    fig.add_trace(
        go.Scatter(x=df['iteration'], y=pulls, name=arm),
        row=2, col=2
    )

fig.update_layout(height=800, showlegend=True, title_text="MAB Experiment Results")
fig.show()